# Section 6: Experiment Viewer（Streamlitアプリ）

## 🎯 このセクションの目標
- **Streamlit in Snowflake** で実験結果を閲覧できるアプリを作成
- 過去の実験のSHAP画像やメトリクスを簡単に確認できるようにする

## 📋 なぜ必要？
- Snowsight UIではアーティファクト画像を直接プレビューできない
- 過去の実験を振り返る際に、毎回ノートブックでコードを書くのは面倒
- → **ワンクリックで過去の実験を可視化**できるアプリがあると便利！

---

## 💡 完成イメージ

```
┌─────────────────────────────────────────────────────────┐
│  🔬 Experiment Viewer                                   │
├─────────────────────────────────────────────────────────┤
│  実験名: [CHURN_PREDICTION_EXPERIMENT ▼]                │
│  Run:    [Baseline ▼]                                   │
├─────────────────────────────────────────────────────────┤
│  📊 メトリクス                                          │
│  ┌────────┬────────┬────────┬────────┐                 │
│  │F1: 0.78│AUC:0.85│Acc:0.82│Time:2s │                 │
│  └────────┴────────┴────────┴────────┘                 │
├─────────────────────────────────────────────────────────┤
│  🖼️ アーティファクト                                    │
│  [Confusion Matrix] [Feature Importance] [SHAP Plot]   │
│                                                         │
│         (選択した画像がここに表示)                       │
└─────────────────────────────────────────────────────────┘
```


---
## 6.1 Streamlitアプリの作成手順

### Step 1: Streamlit App作成画面を開く

1. Snowsight左メニュー → **Projects** → **Streamlit**
2. 右上の **+ Streamlit App** をクリック

### Step 2: アプリ設定

| 項目 | 値 |
|------|---|
| App name | `EXPERIMENT_VIEWER` |
| App location (Database) | `MLOPS_HOL_DB` |
| App location (Schema) | `FEATURE_STORE` |
| App warehouse | `MLOPS_HOL_PYTHON_WH` |

### Step 3: コードを貼り付け

**2つのバージョンがあります：**

| バージョン | 特徴 | 必要な設定 |
|-----------|------|-----------|
| **A. 棒グラフ版** | Altairで可視化 | なし（そのまま動く） |
| **B. 画像表示版** | Notebookで保存したPNG画像を表示 | External Access Integration |

> 💡 **トライアルアカウント**の場合は **A. 棒グラフ版** を使用してください
> 
> 💡 **デモアカウント**等でExternal Accessが使える場合は **B. 画像表示版** も利用可能


In [ ]:
# ============================================================
# 📋 A. 棒グラフ版（パッケージ追加不要・そのまま動く）
# ============================================================

streamlit_code = '''
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd
import json
import altair as alt

# ページ設定
st.set_page_config(
    page_title="Experiment Viewer",
    page_icon="🔬",
    layout="wide"
)

# セッション取得
session = get_active_session()

st.title("🔬 Experiment Viewer")

# データ取得
df = session.table("MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS").to_pandas()

# タブで切り替え
tab1, tab2 = st.tabs(["📊 比較ビュー", "🔍 詳細ビュー"])

# =====================================================
# 比較ビュー（全Run横並び）
# =====================================================
with tab1:
    st.header("📊 全Runの比較")
    
    # メトリクス比較テーブル（過学習チェック付き）
    comparison_df = df[["RUN_NAME", "F1_SCORE", "TRAIN_F1_SCORE", "OVERFIT_GAP_F1", "ROC_AUC", "ACCURACY"]].copy()
    comparison_df.columns = ["Run", "Test F1", "Train F1", "Gap", "ROC-AUC", "Accuracy"]
    st.dataframe(comparison_df, use_container_width=True, hide_index=True)
    
    st.caption("💡 Gap = Train F1 - Test F1（大きいほど過学習の可能性）")
    
    st.divider()
    
    # メトリクス棒グラフ比較
    st.subheader("📈 メトリクス比較")
    
    metrics_data = []
    for _, row in df.iterrows():
        run_name = row["RUN_NAME"].split("_")[0]
        metrics_data.append({"Run": run_name, "Metric": "F1", "Value": row["F1_SCORE"]})
        metrics_data.append({"Run": run_name, "Metric": "ROC-AUC", "Value": row["ROC_AUC"]})
        metrics_data.append({"Run": run_name, "Metric": "Accuracy", "Value": row["ACCURACY"]})
    
    metrics_df = pd.DataFrame(metrics_data)
    
    chart = alt.Chart(metrics_df).mark_bar().encode(
        x=alt.X("Run:N"),
        y=alt.Y("Value:Q", scale=alt.Scale(domain=[0, 1])),
        color=alt.Color("Run:N"),
        column=alt.Column("Metric:N")
    ).properties(width=150, height=300)
    st.altair_chart(chart)
    
    st.divider()
    
    # 特徴量重要度比較
    st.subheader("📈 特徴量重要度 比較")
    
    imp_data = []
    for _, row in df.iterrows():
        run_name = row["RUN_NAME"].split("_")[0]
        importance = json.loads(row["FEATURE_IMPORTANCE"])
        for feat, val in importance.items():
            imp_data.append({"Run": run_name, "Feature": feat, "Importance": val})
    
    imp_df = pd.DataFrame(imp_data)
    
    imp_chart = alt.Chart(imp_df).mark_bar().encode(
        x=alt.X("Importance:Q"),
        y=alt.Y("Feature:N", sort="-x"),
        color=alt.Color("Run:N"),
        row=alt.Row("Run:N")
    ).properties(height=150)
    st.altair_chart(imp_chart, use_container_width=True)

# =====================================================
# 詳細ビュー（1Run詳細）
# =====================================================
with tab2:
    st.header("🔍 Run詳細")
    
    selected_run = st.selectbox("Run名", df["RUN_NAME"].tolist())
    run_data = df[df["RUN_NAME"] == selected_run].iloc[0]
    
    col1, col2, col3, col4, col5 = st.columns(5)
    col1.metric("F1 Score", f"{run_data['F1_SCORE']:.4f}")
    col2.metric("ROC-AUC", f"{run_data['ROC_AUC']:.4f}")
    col3.metric("Accuracy", f"{run_data['ACCURACY']:.4f}")
    col4.metric("Precision", f"{run_data['PRECISION']:.4f}")
    col5.metric("Recall", f"{run_data['RECALL']:.4f}")
    
    st.divider()
    
    col_left, col_right = st.columns(2)
    
    with col_left:
        st.subheader("⚙️ パラメータ")
        st.json(json.loads(run_data['PARAMS']))
    
    with col_right:
        st.subheader("📈 特徴量重要度")
        importance = json.loads(run_data['FEATURE_IMPORTANCE'])
        imp_df = pd.DataFrame({
            "Feature": list(importance.keys()),
            "Importance": list(importance.values())
        }).sort_values("Importance", ascending=False)
        
        chart = alt.Chart(imp_df).mark_bar(color="steelblue").encode(
            x=alt.X("Importance:Q"),
            y=alt.Y("Feature:N", sort="-x")
        ).properties(height=250)
        st.altair_chart(chart, use_container_width=True)
    
    st.divider()
    
    # SHAP値
    st.subheader("🔍 SHAP値（平均絶対値）")
    shap_imp = json.loads(run_data['SHAP_IMPORTANCE'])
    shap_df = pd.DataFrame({
        "Feature": list(shap_imp.keys()),
        "SHAP": list(shap_imp.values())
    }).sort_values("SHAP", ascending=False)
    
    shap_chart = alt.Chart(shap_df).mark_bar(color="coral").encode(
        x=alt.X("SHAP:Q"),
        y=alt.Y("Feature:N", sort="-x")
    ).properties(height=250)
    st.altair_chart(shap_chart, use_container_width=True)
    
    st.divider()
    
    with st.expander("📋 データセット情報"):
        st.json(json.loads(run_data['DATASET_INFO']))

st.caption("🔬 MLOps Hands-on - Experiment Viewer")
'''

print("=" * 70)
print("📋 A. 棒グラフ版（パッケージ追加不要）")
print("=" * 70)
print(streamlit_code)


---
## 6.1.2 B. 画像表示版（External Access必要）

Notebookで保存したSHAP Summary Plot等のPNG画像を表示できます。

### 事前準備（External Access Integration）

以下のSQLをWorksheetで実行してください：

```sql
USE ROLE ACCOUNTADMIN;

CREATE OR REPLACE NETWORK RULE pypi_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('pypi.org', 'files.pythonhosted.org');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION pypi_access_integration
  ALLOWED_NETWORK_RULES = (pypi_network_rule)
  ENABLED = TRUE;

ALTER STREAMLIT MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_VIEWER 
  SET EXTERNAL_ACCESS_INTEGRATIONS = (pypi_access_integration);
```

### pyproject.toml の編集

```toml
[project]
name = "streamlit-app"
requires-python = ">= 3.11"
version = "0.0.1"
dependencies = [
    "streamlit[snowflake]",
    "snowflake-ml-python"
]
```


In [ ]:
# ============================================================
# 📋 B. 画像表示版（External Access Integration必要）
# ============================================================

streamlit_code_with_images = '''
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.ml.experiment import ExperimentTracking
import pandas as pd
import json
import altair as alt
import os

st.set_page_config(page_title="Experiment Viewer", page_icon="🔬", layout="wide")

session = get_active_session()

st.title("🔬 Experiment Viewer")

# データ取得
df = session.table("MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS").to_pandas()

# タブで切り替え
tab1, tab2, tab3 = st.tabs(["📊 比較ビュー", "🔍 詳細ビュー", "🖼️ 画像ビュー"])

# =====================================================
# 比較ビュー
# =====================================================
with tab1:
    st.header("📊 全Runの比較")
    
    comparison_df = df[["RUN_NAME", "F1_SCORE", "TRAIN_F1_SCORE", "OVERFIT_GAP_F1", "ROC_AUC", "ACCURACY"]].copy()
    comparison_df.columns = ["Run", "Test F1", "Train F1", "Gap", "ROC-AUC", "Accuracy"]
    st.dataframe(comparison_df, use_container_width=True, hide_index=True)
    
    st.caption("💡 Gap = Train F1 - Test F1（大きいほど過学習の可能性）")
    
    st.divider()
    
    st.subheader("📈 メトリクス比較")
    metrics_data = []
    for _, row in df.iterrows():
        run_name = row["RUN_NAME"].split("_")[0]
        metrics_data.append({"Run": run_name, "Metric": "F1", "Value": row["F1_SCORE"]})
        metrics_data.append({"Run": run_name, "Metric": "ROC-AUC", "Value": row["ROC_AUC"]})
        metrics_data.append({"Run": run_name, "Metric": "Accuracy", "Value": row["ACCURACY"]})
    
    metrics_df = pd.DataFrame(metrics_data)
    chart = alt.Chart(metrics_df).mark_bar().encode(
        x=alt.X("Run:N"),
        y=alt.Y("Value:Q", scale=alt.Scale(domain=[0, 1])),
        color=alt.Color("Run:N"),
        column=alt.Column("Metric:N")
    ).properties(width=150, height=300)
    st.altair_chart(chart)

# =====================================================
# 詳細ビュー
# =====================================================
with tab2:
    st.header("🔍 Run詳細")
    
    selected_run = st.selectbox("Run名", df["RUN_NAME"].tolist(), key="detail_run")
    run_data = df[df["RUN_NAME"] == selected_run].iloc[0]
    
    col1, col2, col3, col4, col5 = st.columns(5)
    col1.metric("F1 Score", f"{run_data['F1_SCORE']:.4f}")
    col2.metric("ROC-AUC", f"{run_data['ROC_AUC']:.4f}")
    col3.metric("Accuracy", f"{run_data['ACCURACY']:.4f}")
    col4.metric("Precision", f"{run_data['PRECISION']:.4f}")
    col5.metric("Recall", f"{run_data['RECALL']:.4f}")
    
    st.divider()
    
    col_left, col_right = st.columns(2)
    with col_left:
        st.subheader("⚙️ パラメータ")
        st.json(json.loads(run_data['PARAMS']))
    
    with col_right:
        st.subheader("📈 特徴量重要度")
        importance = json.loads(run_data['FEATURE_IMPORTANCE'])
        imp_df = pd.DataFrame({
            "Feature": list(importance.keys()),
            "Importance": list(importance.values())
        }).sort_values("Importance", ascending=False)
        chart = alt.Chart(imp_df).mark_bar(color="steelblue").encode(
            x=alt.X("Importance:Q"),
            y=alt.Y("Feature:N", sort="-x")
        ).properties(height=250)
        st.altair_chart(chart, use_container_width=True)

# =====================================================
# 画像ビュー（アーティファクト）
# =====================================================
with tab3:
    st.header("🖼️ アーティファクト画像")
    
    exp = ExperimentTracking(session=session)
    exp.set_experiment("CHURN_PREDICTION_EXPERIMENT")
    
    selected_run_img = st.selectbox("Run名", df["RUN_NAME"].tolist(), key="image_run")
    display_name = selected_run_img.split("_")[0]
    
    image_type = st.radio(
        "表示する画像",
        ["Confusion Matrix", "Feature Importance", "SHAP Summary Plot"],
        horizontal=True
    )
    
    artifact_map = {
        "Confusion Matrix": f"confusion_matrix_{display_name}.png",
        "Feature Importance": f"feature_importance_{display_name}.png",
        "SHAP Summary Plot": f"shap_summary_{display_name}.png"
    }
    
    artifact_name = artifact_map[image_type]
    download_path = "/tmp/viewer/"
    os.makedirs(download_path, exist_ok=True)
    
    if st.button(f"📥 {image_type} を表示", type="primary"):
        with st.spinner("画像を取得中..."):
            try:
                exp.download_artifacts(
                    run_name=selected_run_img,
                    artifact_path=artifact_name,
                    target_path=download_path
                )
                image_path = f"{download_path}{artifact_name}"
                if os.path.exists(image_path):
                    st.image(image_path, caption=f"{image_type} - {display_name}")
                    st.success("✅ 画像を表示しました")
                else:
                    st.warning("画像が見つかりません")
            except Exception as e:
                st.error(f"エラー: {e}")

st.caption("🔬 MLOps Hands-on - Experiment Viewer")
'''

print("=" * 70)
print("📋 B. 画像表示版（External Access Integration必要）")
print("=" * 70)
print(streamlit_code_with_images)


---
## 6.2 アプリの機能解説

### 📊 メトリクス表示
```python
# EXPERIMENT_RESULTSテーブルから取得
st.metric("F1 Score", f"{run_data['F1_SCORE']:.4f}")
```
- Section 4で保存したテーブルから直接読み込み
- リアルタイムで最新データを表示

### 📈 特徴量重要度グラフ
```python
# JSONをパースして棒グラフ化
importance = json.loads(run_data['FEATURE_IMPORTANCE'])
```
- Altairで動的なグラフを生成
- 重要度順にソートして表示

### 🖼️ アーティファクト画像
```python
# Experiment Trackingからダウンロード
exp.download_artifacts(
    run_name=selected_run,
    artifact_path=artifact_name,
    target_path=download_path
)
st.image(image_path)
```
- ボタンクリックでオンデマンドダウンロード
- 必要な画像だけ取得するので軽量


---
## ✅ Section 6 完了！

### 学んだこと
- **Streamlit in Snowflake** でアプリを作成
- Experiment Trackingのデータを可視化
- アーティファクト（画像）をアプリ内で表示

### 作成したもの
- Streamlit App: `EXPERIMENT_VIEWER`
  - 実験・Run選択機能
  - メトリクス表示（F1, AUC, Accuracy等）
  - 特徴量重要度グラフ
  - SHAP値グラフ
  - アーティファクト画像表示

### 💡 活用シーン
- **実験レビュー**: 過去の実験結果をチームで共有
- **モデル選定**: 複数モデルの比較検討
- **監査対応**: モデルの説明責任を果たす
- **ドキュメント**: スクリーンショットでレポート作成

---

## 🎉 ハンズオン完了！

お疲れさまでした！以下のMLOpsコンポーネントを体験しました：

| Section | 内容 |
|---------|------|
| 01 | 環境構築・データ探索 |
| 02 | Feature Store |
| 03 | Model Training |
| 04 | Experiment Tracking |
| 05 | Model Registry & 推論 |
| 06 | Experiment Viewer (Streamlit) |

これらを組み合わせることで、**本番運用可能なMLOpsパイプライン**を構築できます！
